In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
from pandas import Series, DataFrame
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta  

In [3]:
#pretend to be a browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [4]:
#grand tour urls
#giro d-italia
url_gdi = 'https://www.procyclingstats.com/race/giro-d-italia/'
#tour de france
url_tdf = 'https://www.procyclingstats.com/race/tour-de-france/'
#vuelta a espana
url_vae = 'https://www.procyclingstats.com/race/vuelta-a-espana/'

In [5]:
#url data
base_url = 'https://www.procyclingstats.com/race/'

In [6]:
#grand tours
tours = ['giro-d-italia','tour-de-france','vuelta-a-espana']
#considered years
years = list(range(2010,2017,1))
#stages
stages = list(range(1,21,1))

In [73]:
#Pull grand tour data
tour = tours[0]
year = str(years[0])
stage = str(stages[0])
url = base_url + tour +'/'+ year + '/stage-' + stage
r = requests.get(url,headers = header)
results = pd.read_html(r.text)
results = results[0]

#add fields
results['race'] = tour
results['year'] = year
results['stage'] = stage
results.columns = results.columns.str.lower()

In [74]:
#fix time
time = [x[0] for x in results.time.str.split()]

for n,t in enumerate(time):
    if t == ',,':
        time[n] = '00:00:00'
    elif len(t) == 4:
        time[n] = '00:0' + t
    elif len(t) == 5:
        time[n] = '00:' + t
    elif len(t) == 7:
        time[n] = '0' + t

time = pd.to_datetime(time,format='%H:%M:%S')    
results.time = Series(time).dt.time

In [75]:
results.head()

,rnk,bib,rider,age,team,uci,pnt,avg,time,gc,gc-time,race,year,stage
0,1,171,Wiggins Bradley,30,Sky Procycling,16.0,80.0,48.932,00:10:18,1.0,+0:00,giro-d-italia,2010,1
1,2,2,Bookwalter Brent,26,BMC Racing Team,8.0,50.0,48.774,00:00:02,NaN,+0:00,giro-d-italia,2010,1
2,3,1,Evans Cadel,33,BMC Racing Team,4.0,35.0,48.774,00:00:00,NaN,+0:00,giro-d-italia,2010,1
3,4,41,Vinokourov Alexandre,36,Astana,2.0,25.0,48.539,00:00:05,NaN,+0:00,giro-d-italia,2010,1
4,5,177,Henderson Gregory,33,Sky Procycling,1.0,18.0,48.539,00:00:00,NaN,+0:00,giro-d-italia,2010,1


In [76]:
cum_time = []
for n,t in enumerate(time):
    if n == 0:
        cum_time.append(time[n])
    else:
        hours = time[n].hour
        minutes = time[n].minute
        seconds = time[n].second
        ct = cum_time[n-1] + timedelta(hours = hours, minutes = minutes, seconds = seconds)
        cum_time.append(ct)
results['cum_time'] = Series(cum_time).dt.time

In [77]:
results.head()

,rnk,bib,rider,age,team,uci,pnt,avg,time,gc,gc-time,race,year,stage,cum_time
0,1,171,Wiggins Bradley,30,Sky Procycling,16.0,80.0,48.932,00:10:18,1.0,+0:00,giro-d-italia,2010,1,00:10:18
1,2,2,Bookwalter Brent,26,BMC Racing Team,8.0,50.0,48.774,00:00:02,NaN,+0:00,giro-d-italia,2010,1,00:10:20
2,3,1,Evans Cadel,33,BMC Racing Team,4.0,35.0,48.774,00:00:00,NaN,+0:00,giro-d-italia,2010,1,00:10:20
3,4,41,Vinokourov Alexandre,36,Astana,2.0,25.0,48.539,00:00:05,NaN,+0:00,giro-d-italia,2010,1,00:10:25
4,5,177,Henderson Gregory,33,Sky Procycling,1.0,18.0,48.539,00:00:00,NaN,+0:00,giro-d-italia,2010,1,00:10:25
